In [ ]:
#解压数据集
!unzip data/data152113/signs.zip

In [ ]:
#导入库
import io
import os
import paddle
from PIL import Image
import paddle.io as pio
import paddle.vision.transforms as pt
import paddle.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 导入库后，第一次运行需要标签标注，标注完成后，不再需要运行此代码块

In [ ]:
#标签列表
LABEL_MAP=["ratt","ox","tiger","rabbit","dragon","snake","horse","goat","monkey","rooster","dog","pig"]
#根目录
root = 'signs'
#标注生成函数
def Annotation(mode):
    #先创建标注文件
    with open('{}/{}.txt'.format(root,mode),'w') as f:
        #训练，验证，测试三种文件夹路径
        data_dir = '{}/{}'.format(root,mode)
        #遍历文件夹,listdir函数可以返回data_dir下所有文件与文件夹列表
        for path in os.listdir(data_dir):
            #生成标签索引
            label_index = LABEL_MAP.index(path)
            #图像样本路径
            img_path = '{}/{}'.format(data_dir,path)
            #遍历所有图像样本
            for image in os.listdir(img_path):
                #单个图像的样本路径
                image_file = '{}/{}'.format(img_path,image)
                try:
                    with open(image_file,'rb') as f_img:
                        # io.BytesIO将字节对象转为Byte字节流数据,供Image.open使用
                        #f_img.read([size]) 将文件数据作为字符串返回，可选参数size控制读取的字节数
                        image=Image.open(io.BytesIO(f_img.read()))
                        #调用load()方法将强行加载图像数据
                        image.load()
                        if image.mode=='RGB':
                            #写入创建好的text文件
                            f.write('{}\t{}\n'.format(image_file,label_index))
                except:
                    continue
#生成三个数据集标注文件
Annotation(mode = 'train')
print('train annotation file created')
Annotation(mode = 'valid')
print('valid annotation file created')
Annotation(mode = 'test')
print('test annotation file created')

# 自定义数据集

In [ ]:
#自定义数据集
class MyDataset(pio.Dataset):
    def __init__(self, mode = 'train'):
        super(MyDataset, self).__init__()
        #判断mode参数
        assert mode in ['train', 'test', 'valid'],'mode is one of train, test, valid'
        self.data = []
        #读取对应mode的索引记录
        with open('signs/{}.txt'.format(mode)) as f:
            #按行读取
            for line in f.readlines():
                #去除首尾制表符
                info = line.strip().split('\t')
                if len(info) > 0:
                    #将读取信息添加到data列表中
                    self.data.append([info[0].strip(),info[1].strip()])
        #定义预处理方法，增加样本多样性
        if mode == 'train':
            self.transforms = pt.Compose([
                pt.RandomResizedCrop(size = 224),#随机裁剪大小，从原图裁剪为224*224
                pt.RandomHorizontalFlip(0.5),#以0.5的概率随机水平翻转
                pt.ToTensor(), #转变为张量模式
                pt.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]) #归一化
            ])
        else:
            #评估不需要增加样本多样性
            self.transforms=pt.Compose([
                pt.Resize(size=256),#先压缩到256*256
                pt.RandomCrop(size=224),
                pt.ToTensor(),
                pt.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])#归一化
            ])
    def __getitem__(self,index):
        #获取单个样本数据和标签
        image_file,label = self.data[index]
        image = Image.open(image_file)
        #非RGB格式图像转化为RGB格式
        if image.mode != 'RGB':
            image = image.convert('RGB')
        #图像预处理
        image = self.transforms(image)
        #将标签转换为numpy形式
        return image,np.array(label, dtype = 'int64')
    def __len__(self):
        return (len(self.data))

# 创建数据集实例


In [ ]:
#创建数据集实例
train_data = MyDataset(mode = 'train')
test_data = MyDataset(mode = 'test')
valid_data = MyDataset(mode = 'valid')
print ('训练集数目: {}, 验证集数目: {}, 测试集数目: {}'.format(len(train_data),len(valid_data),len(test_data)))

# VGG16网络组网

In [ ]:
#模型组网VGG-16
class VGG(nn.Layer):
    def __init__(self):
        super(VGG,self).__init__()
        self.act = nn.ReLU()
        #第一部分
        self.c1_1=nn.Conv2D(3,64,3,1,1)
        self.c1_2=nn.Conv2D(64,64,3,1,1)
        self.m1=nn.MaxPool2D(kernel_size=2,stride=2)
        #第二部分
        self.c2_1=nn.Conv2D(64,128,3,1,1)
        self.c2_2=nn.Conv2D(128,128,3,1,1)
        self.m2=nn.MaxPool2D(kernel_size=2,stride=2)
        #第三部分
        self.c3_1=nn.Conv2D(128,256,3,1,1)
        self.c3_2=nn.Conv2D(256,256,3,1,1)
        self.c3_3=nn.Conv2D(256,256,3,1,1)
        self.m3=nn.MaxPool2D(kernel_size=2,stride=2)
        #第四部分
        self.c4_1=nn.Conv2D(256,512,3,1,1)
        self.c4_2=nn.Conv2D(512,512,3,1,1)
        self.c4_3=nn.Conv2D(512,512,3,1,1)
        self.m4=nn.MaxPool2D(kernel_size=2,stride=2)
        #第五部分
        self.c5_1=nn.Conv2D(512,512,3,1,1)
        self.c5_2=nn.Conv2D(512,512,3,1,1)
        self.c5_3=nn.Conv2D(512,512,3,1,1)
        self.m5=nn.MaxPool2D(kernel_size=2,stride=2)
        self.fl=nn.Flatten()
        self.fc1=nn.Linear(in_features=512*7*7,out_features=4096)
        self.fc2=nn.Linear(in_features=4096,out_features=4096)
        self.fc3=nn.Linear(in_features=4096,out_features=12)
        self.d=nn.Dropout()
    
    def forward(self,inputs):
        x=self.c1_1(inputs)
        x=self.act(x)
        x=self.c1_2(x)
        x=self.act(x)
        x=self.m1(x)
       
        x=self.c2_1(x)
        x=self.act(x)
        x=self.c2_2(x)
        x=self.act(x)
        x=self.m2(x)

        x=self.c3_1(x)
        x=self.act(x)
        x=self.c3_2(x)
        x=self.act(x)
        x=self.c3_3(x)
        x=self.act(x)
        x=self.m3(x)

        x=self.c4_1(x)
        x=self.act(x)
        x=self.c4_2(x)
        x=self.act(x)
        x=self.c4_3(x)
        x=self.act(x)
        x=self.m4(x)

        x=self.c5_1(x)
        x=self.act(x)
        x=self.c5_2(x)
        x=self.act(x)
        x=self.c5_3(x)
        x=self.act(x)
        x=self.m5(x)

        #接分类层
        x=self.fl(x)

        x=self.fc1(x)
        x=self.act(x)
        x=self.d(x)

        x=self.fc2(x)
        x=self.act(x)
        x=self.d(x)

        x=self.fc3(x)
        return x
model = VGG()
paddle.summary(model,(1,3,224,224))

# 开始训练，首先调整参数

In [ ]:
#超参设置
batch_size = 64
epochs = 5
learning_rate = 0.0001
#损失函数:cross_entropy为交叉熵损失，nll为negative log likelihood loss损失
loss_function = 'cross_entropy'

#开始训练
train_d = paddle.io.DataLoader(train_data, batch_size=batch_size,shuffle=True,drop_last=True)
valid_d = paddle.io.DataLoader(valid_data, batch_size=batch_size,shuffle=True,drop_last=True)
def train(model,epochs,learning_rate,loss_function,batch_size):
    use_gpu = True
    #资源配置
    paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

    cross_entropy = paddle.nn.CrossEntropyLoss()
    log_softmax = paddle.nn.LogSoftmax(axis = 1)
    nll = paddle.nn.NLLLoss()

    #用正则化来提升精度
    #opt = paddle.optimizer.Adam(learning_rate=learning_rate,parameters=model.parameters())
    opt = paddle.optimizer.Momentum(learning_rate=learning_rate,parameters=model.parameters(),weight_decay=0.0001)
    iter1 = 0
    iters = []
    losses = []
    acces = []
    model.train()
    #进行数据记录
    for epoch in range(epochs):
        for batch_id,data in enumerate(train_d()):
            image = data[0]
            label = data[1]
            label = paddle.reshape(label,(batch_size,1))
            label_1 = label
            predict = model (image)
            if loss_function == 'cross_entropy':
                loss = cross_entropy(predict,label)
            elif loss_function == 'nll':
                #predict = paddle.reshape(predict,(batch_size,1))
                #predict = paddle.cast(predict,dtype='float32')
                label = paddle.cast(label,dtype='int64')
                predict = log_softmax(predict)
                loss = nll(predict,label)
            else:
                print('wrong loss function')
                exit
            avg_loss=paddle.mean(loss)
            predict_1=nn.functional.softmax(predict)
            acc=paddle.metric.accuracy(predict_1,label_1)
            
            if batch_id % 10 ==0:
                print('epoch_id:{}, batch_id:{}, train_loss:{}, train_acc:{}'.format(epoch,batch_id,avg_loss.numpy(),acc.numpy()))
                iters.append(iter1)
                acces.append(acc.numpy())
                losses.append(avg_loss.numpy())
                iter1 = iter1 + 10
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        model.eval()
        total_acc = []
        total_loss = []
        #验证数据
        for batch_id,data in enumerate(valid_d()):
            v_image=data[0]
            v_label=data[1]
            v_label=paddle.reshape(v_label,(batch_size,1))
            v_label_1 = v_label
            v_predict=model(v_image)
            if loss_function == 'cross_entropy':
                v_loss = nn.functional.cross_entropy(v_predict,v_label)
            elif loss_function == 'nll':
                #v_predict = paddle.reshape(predict,(batch_size,1))
                #v_predict = paddle.cast(predict,dtype='float32')
                v_label = paddle.cast(v_label,dtype='int64')
                v_predict = log_softmax(v_predict)
                v_loss = nll(v_predict,v_label)
            else:
                print('wrong loss function')
                exit
            v_avg_loss=paddle.mean(v_loss)
            v_predict_1=nn.functional.softmax(v_predict)
            v_acc=paddle.metric.accuracy(v_predict_1,v_label_1)
            v_avg_loss.backward()
            opt.step()
            opt.clear_grad()
            total_loss.append(v_avg_loss.numpy())
            total_acc.append(v_acc.numpy())
        mean_loss=np.mean(total_loss)
        mean_acc=np.mean(total_acc)
        print('epoch_id:{},  valid_loss:{},  valid_acc:{}  '.format(epoch,mean_loss,mean_acc))
        model.train()
    return iters, losses, acces
model = VGG()
#恢复训练
params_file_path = './VGG16_0613.pdparams'
param_dict = paddle.load(params_file_path)
model.load_dict(param_dict)
iters,losses,acces=train(model,epochs,learning_rate,loss_function,batch_size)
paddle.save(model.state_dict(),'./VGG16_0615.pdparams')


# 测试训练模型

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,classification_report
import pandas as pd
import seaborn as sns

params_file_path = './VGG16_0615.pdparams'
param_dict = paddle.load(params_file_path)
model.load_dict(param_dict)
test_d=paddle.io.DataLoader(test_data,batch_size=32,shuffle=True,drop_last=True)
y_p = []
y_t = []
for batch, (test_X,test_y) in enumerate(test_d):
    y_pred = model(test_X)
    for i in y_pred:
        y_p.append(np.argmax(i))
    #y_p.append(y_pred)
    for i in test_y:
        y_t.append(int(i))

cm = confusion_matrix(y_t,y_p)
cm = pd.DataFrame(cm,columns=["ratt","ox","tiger","rabbit","dragon","snake","horse","goat","monkey","rooster","dog","pig"],index=["ratt","ox","tiger","rabbit","dragon","snake","horse","goat","monkey","rooster","dog","pig"])
sns.heatmap(cm,cmap="YlGnBu_r",fmt="d",annot=True)

print('f1_score:',f1_score(y_t,y_p,average='macro'))
print('accuracy:',accuracy_score(y_t,y_p))
print('error rate:',1-accuracy_score(y_t,y_p))
print('precision:',precision_score(y_t,y_p,average='macro'))
print('recall:',recall_score(y_t,y_p,average='macro'))

# 模型预测

In [ ]:
LABEL_MAP=["rat","ox","tiger","rabbit","dragon","snake","horse","goat","monkey","rooster","dog","pig"]
#test文件夹进行测试模型
root1='test'
#将图片路径写入txt文本文件
with open('{}.txt'.format(root1),'w') as f:
    for image in os.listdir(root1):
        #单个图像样本路径
        image_file1='{}/{}'.format(root1,image)
        f.write('{}\n'.format(image_file1))
#定义预处理方式
def transform(img):
    if img.mode !='RGB':
        img=img.convert('RGB')
    transforms=pt.Compose([
                pt.Resize(size=256),#先压缩到256*256
                pt.RandomCrop(size=224),
                pt.ToTensor(),
                pt.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])#归一化
            ])
    img=transforms(img)
    img=paddle.reshape(img,(1,3,224,224))
    return img

#将创建好的文本中图片路径写入一个列表
test_list=[]
with open('test.txt') as f:
    img=f.readlines()
    for i in range(len(img)):
        test_list.append(img[i].strip())
    print(test_list)
#加载模型
model=VGG()
params_file_path = './VGG16_0615.pdparams'
param_dict = paddle.load(params_file_path)
model.load_dict(param_dict)
model.eval()
for i in range(len(test_list)):
    img=Image.open(test_list[i])
    plt.figure()
    plt.imshow(img)
    img=transform(img)
    result=model(img)
    result=paddle.argmax(result[0])
    plt.title('predict:{}'.format(LABEL_MAP[result]))
    plt.axis('on')
    plt.show()